# 基于 MindQuantum 0.7.0 实现半导体双量子点下的双量子比特 Grover 算法。

In [1]:
from mindquantum import *

circ = Circuit()
circ += H.on(0)
circ += H.on(1)
circ += Z.on(1,0)
circ += H.on(0)
circ += H.on(1)
circ += Z.on(0)
circ += Z.on(1)
circ += Z.on(1,0)
circ += H.on(0)
circ += H.on(1)

sim = Simulator('projectq', 2)
sim.apply_circuit(circ)
print(sim.get_qs(ket=True))

1¦11⟩


In [16]:
import copy
import numpy as np
import mindspore as ms
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
from mindspore.ops import operations
from mindspore import nn, ops, Tensor, context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
from mindspore.nn import Adam, TrainOneStepCell, LossBase
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

def _matrix_(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_(coeff):
    return -1j*_matrix_(coeff)@(s_z*dt)

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*ddt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*ddt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10

cz_params = np.array([1.5472503,  1.4179231,  1.540713,   1.9724044,  1.9253408,  1.3879265,
                     0.8130467,  0.76446086, 1.2703444,  1.8553745,  1.0291328,  1.2492974,
                     0.7880994,  0.3026381,  0.31203356, 0.30834132, 0.9533752,  1.3802187,
                     1.270656,   0.5646567,  0.94619316, 0.97377133, 1.9658349,  0.83277696,
                     1.0190777,  0.90001523, 0.26008993, 0.16526282, 0.22249524, 1.1596956,
                     1.5285202,  0.4919534,  0.01645389, 0.02608137, 0.6504683,  0.31325826,
                     0.4486266,  0.8677286,  1.3571227,  1.4995408,  1.1248059,  0.5996333,
                     0.32797617, 0.54987127])

h_params = np.array([0.93665886, 0.24112344, 3.1801345,  1.1307619,  1.5094017,  3.022603,
                     0.45939285, 2.7383144,  1.6714032,  2.1061618,  2.3177178,  1.2582982])

z_params = np.array([1.6117827,  1.3258268,  0.0498501,  2.3190951,  0.27652496, 1.5626596,
                     0.70895815, 0.61659425, 1.8357016,  1.4761814,  2.117067,   0.6488876 ])

def cz_circ():
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(param).on(0) for param in cz_params[:10]])
    circ_ += Circuit([gate_0(0).on(1) for i in range(10)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(1) for param in cz_params[10:20]])
    circ_ += Circuit([gate_c_0(cz_params[20]).on([1,0]), gate_c_0(cz_params[21]).on([1,0])])
    circ_ += Circuit([gate_c_1(cz_params[22]).on([1,0]), gate_c_1(cz_params[23]).on([1,0])])
    circ_ += Circuit([gate_0(param).on(0) for param in cz_params[24:34]])
    circ_ += Circuit([gate_0(0).on(1) for i in range(10,20)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(1) for param in cz_params[34:]])
    circ_ += BarrierGate()
    return circ_

def i_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(0).on(qubit)
    return circ_

def h_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(h_params[i]).on(qubit)
    return circ_

def z_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(z_params[i]).on(qubit)
    return circ_

circ = Circuit()

circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(0)
circ += h_circ(1)

circ += cz_circ()

circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(0)
circ += h_circ(1)

circ += z_circ(0)
circ += i_circ(1)
circ += i_circ(0)
circ += z_circ(1)

circ += cz_circ()

circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(0)
circ += h_circ(1)


sim = Simulator('projectq', 2)
sim.apply_circuit(circ)
state = sim.get_qs()
distribution = []
for i in range(len(state)):
    distribution.append((state[i].conj()*state[i]).real)
print(distribution)

[2.6125416398212854e-05, 3.064878890397258e-05, 3.449741938457886e-05, 0.9999087283753169]
